In [ ]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

# Week 3: Lecture 6 Census

## Table Review: Welcome Survey

Complete this [Class Data Survey](https://forms.gle/ZkheGLg8LETiiNvT8). Individuals will not be identified. You may leave any answers blank. We'll use this Class Data to use the rest of the quarter. Thank you in advance!

Please complete the survey by Tues, April 19. Once you've completed it, please take a screenshot of the confirmation message and submit your screenshot to Gradescope.

## Discussion Question: NBA Salaries

In [ ]:
nba = Table.read_table('data/nba_salaries.csv')
nba = nba.relabeled(3, 'SALARY').drop('TEAM')
nba.show(3)

In [ ]:
# Question (a)




In [ ]:
# Question (b)




## Census: Population Trends ##

In [ ]:
# As of August 2021, this census file is online here: 
data = 'http://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2019/nc-est2019-agesex-res.csv'

# A description of the table appears online.
# https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2019/nc-est2019-agesex-res.pdf

full_census_table = Table.read_table(data)
full_census_table

[A description of the table appears online.](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2019/nc-est2019-agesex-res.pdf)

The Census dataset contain estimates of the US population in each category of sex and age in the years 2010 through 2019. The Census is decennial: it takes place every 10 years. The most recent Census was held in 2020 and the one before that in 2010. The Census Bureau also estimates population changes each year. As explained in the Bureau’s description of its methodology, it “adds [the estimated changes] to the last decennial census to produce updated population estimates every year.”



In [ ]:
partial_census_table = full_census_table.select('SEX', 'AGE', 'POPESTIMATE2014', 'POPESTIMATE2019')
partial_census_table.sort("SEX",descending=True)

In [ ]:
us_pop = partial_census_table.relabeled('POPESTIMATE2014', '2014').relabeled('POPESTIMATE2019', '2019')
us_pop

In [ ]:
us_pop_by_age = us_pop.where('SEX', are.equal_to(0)).drop('SEX')
us_pop_by_age


# what does SEX = 0 stand for?

In [ ]:
us_pop_by_age.where('AGE', are.between(97, 101))

In [ ]:
change = us_pop_by_age.column('2019') - us_pop_by_age.column('2014')
change

In [ ]:
us_pop_change = us_pop_by_age.with_columns(
    'Change', change,
    'Percent Change', change/us_pop_by_age.column('2014')
)
us_pop_change.set_format('Percent Change', PercentFormatter)

In [ ]:
us_pop_change.where('AGE', are.equal_to(999))
# what does age 999 stand for?





In [ ]:
us_pop_change.where(
    'AGE', are.below(999)
).sort('Change', descending=True)

The Census form asks respondents to provide the sex of each household member by checking one of two boxes labeled Male and Female. The SEX column contains numeric codes: 1 for male, 2 for female, and 0 for the total.



Take a look at the top few rows. While the percent change is about 3% for the overall population, it jumps to well over 20% for the people in their late sixties and early seventies. This stunning change contributes to what is known as the greying of America.

What could explain this large increase? Take a moment to write down your thoughts below. After, please discuss with your neighbor/table.

## Sex Ratios

The Census form asks respondents to provide the sex of each household member by checking one of two boxes labeled Male and Female. The SEX column contains numeric codes: 1 for male, 2 for female, and 0 for the total.



In [ ]:
us_pop_2019 = us_pop.drop('2014')
all_ages = us_pop_2019.where('AGE', are.equal_to(999))
all_ages

In [ ]:
pop_2019 = all_ages.column('2019').item(0)
all_ages.with_column(
    'Proportion', all_ages.column('2019')/pop_2019
).set_format('Proportion', PercentFormatter)

In [ ]:
infants = us_pop_2019.where('AGE', are.equal_to(0))
infants

In [ ]:
infants_2019 = infants.column('2019').item(0)
infants.with_column(
    'Proportion', infants.column('2019')/infants_2019
).set_format('Proportion', PercentFormatter)

In [ ]:
females_all_rows = us_pop_2019.where('SEX', are.equal_to(2))
females = females_all_rows.where('AGE', are.not_equal_to(999))
females

In [ ]:
males_all_rows = us_pop_2019.where('SEX', are.equal_to(1))
males = males_all_rows.where('AGE', are.not_equal_to(999))
males

In [ ]:
males.column('AGE')

In [ ]:
females.column('AGE')

In [ ]:
ratios = Table().with_columns(
    'AGE', females.column('AGE'),
    '2019 F:M RATIO', females.column('2019')/males.column('2019')
)
ratios

In [ ]:
# let's look at just the 2019 F:M ratios for ages 76 and above:





In [ ]:
males.where('AGE', are.contained_in(make_array(92, 93, 99)))

In [ ]:
# now let's look at the 2019 female population for 92, 93 and 99 year olds:





In [ ]:
# what about the 2019 female population for 100-105 year olds?





## Visualization ##

In [ ]:
ratios.plot('AGE')